In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [7]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [8]:
subject = 'Sergipe - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [9]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_SE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Sergipe - Produção de Cimento (t)', axis=1)
data

,Ano,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao,Sergipe - Consumo de Cimento (t)
0,2008,2.275243e+07,1.304488e+06,9.995682,2.046863e+07,0.684816,0.671003,5.483335e+08,2.546644e+09,1.874883e+10,...,8.151229,12.692687,1013.229689,2586.438280,26.686275,23.696582,79.520472,3.343797,0.620749,391.414000
1,2009,2.284202e+07,1.307162e+06,9.962429,2.041487e+07,0.686798,0.671628,5.302764e+08,2.679945e+09,2.055015e+10,...,8.181780,14.565587,1013.167328,1929.165982,26.816394,23.060223,77.670883,3.361346,0.624297,404.787000
2,2010,2.337741e+07,1.316094e+06,9.971938,2.052469e+07,0.688421,0.671874,5.046425e+08,2.891930e+09,2.342247e+10,...,8.145498,16.790973,1013.363227,1655.291909,26.715685,21.890153,74.115505,3.285878,0.627591,480.282000
3,2011,2.381828e+07,1.308563e+06,9.960908,2.056187e+07,0.689694,0.671748,4.854622e+08,3.091988e+09,2.604265e+10,...,8.132819,17.733809,1013.713445,1515.293481,26.636305,20.489803,70.722822,3.172559,0.630627,579.482000
4,2012,2.420222e+07,1.297619e+06,9.936855,2.057944e+07,0.690538,0.671098,4.867661e+08,3.189506e+09,2.708753e+10,...,11.832646,19.761559,1013.891804,1451.567020,26.670892,19.276376,68.444205,3.096016,0.633403,646.084000
5,2013,2.449152e+07,1.275643e+06,9.895912,2.055335e+07,0.691083,0.670440,5.225848e+08,3.093872e+09,2.523394e+10,...,12.530554,21.193985,1014.116250,1407.100074,26.696079,19.374702,68.963603,3.006992,0.635603,590.339000
6,2014,2.470591e+07,1.254702e+06,9.861955,2.051042e+07,0.691757,0.669226,5.676413e+08,2.779340e+09,1.954361e+10,...,13.099831,20.010392,1014.355116,1389.188275,26.773639,19.486560,67.782988,2.799381,0.638737,633.190143
7,2015,2.484109e+07,1.238716e+06,9.830307,2.046045e+07,0.691403,0.667662,5.179392e+08,2.561731e+09,1.327596e+10,...,13.605605,18.966088,1014.406517,1396.519093,26.838598,19.578296,66.497486,2.216394,0.639881,592.014514
8,2016,2.492621e+07,1.222457e+06,9.814014,2.043197e+07,0.700162,0.672766,4.524144e+08,2.516859e+09,1.011370e+10,...,13.929548,18.086986,1014.355535,1396.331304,26.770657,19.675461,66.457655,1.943788,0.650966,552.624343
9,2017,2.497554e+07,1.205785e+06,9.822126,2.044513e+07,0.700449,0.671836,4.213689e+08,2.497710e+09,8.456309e+09,...,13.725374,18.116372,1014.300674,1352.042945,26.799279,19.803363,66.478178,2.248768,0.653326,473.789000


In [10]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
0,2.275243e+07,1.304488e+06,9.995682,2.046863e+07,0.684816,0.671003,5.483335e+08,2.546644e+09,1.874883e+10,2.351319e+10,...,0.769402,8.151229,12.692687,1013.229689,2586.438280,26.686275,23.696582,79.520472,3.343797,0.620749
1,2.284202e+07,1.307162e+06,9.962429,2.041487e+07,0.686798,0.671628,5.302764e+08,2.679945e+09,2.055015e+10,2.446518e+10,...,0.771521,8.181780,14.565587,1013.167328,1929.165982,26.816394,23.060223,77.670883,3.361346,0.624297
2,2.337741e+07,1.316094e+06,9.971938,2.052469e+07,0.688421,0.671874,5.046425e+08,2.891930e+09,2.342247e+10,2.606035e+10,...,0.773103,8.145498,16.790973,1013.363227,1655.291909,26.715685,21.890153,74.115505,3.285878,0.627591
3,2.381828e+07,1.308563e+06,9.960908,2.056187e+07,0.689694,0.671748,4.854622e+08,3.091988e+09,2.604265e+10,2.845738e+10,...,0.774160,8.132819,17.733809,1013.713445,1515.293481,26.636305,20.489803,70.722822,3.172559,0.630627
4,2.420222e+07,1.297619e+06,9.936855,2.057944e+07,0.690538,0.671098,4.867661e+08,3.189506e+09,2.708753e+10,3.181492e+10,...,0.774664,11.832646,19.761559,1013.891804,1451.567020,26.670892,19.276376,68.444205,3.096016,0.633403
5,2.449152e+07,1.275643e+06,9.895912,2.055335e+07,0.691083,0.670440,5.225848e+08,3.093872e+09,2.523394e+10,3.629162e+10,...,0.774636,12.530554,21.193985,1014.116250,1407.100074,26.696079,19.374702,68.963603,3.006992,0.635603
6,2.470591e+07,1.254702e+06,9.861955,2.051042e+07,0.691757,0.669226,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,0.774272,13.099831,20.010392,1014.355116,1389.188275,26.773639,19.486560,67.782988,2.799381,0.638737
7,2.484109e+07,1.238716e+06,9.830307,2.046045e+07,0.691403,0.667662,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,0.773462,13.605605,18.966088,1014.406517,1396.519093,26.838598,19.578296,66.497486,2.216394,0.639881
8,2.492621e+07,1.222457e+06,9.814014,2.043197e+07,0.700162,0.672766,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,0.784003,13.929548,18.086986,1014.355535,1396.331304,26.770657,19.675461,66.457655,1.943788,0.650966
9,2.497554e+07,1.205785e+06,9.822126,2.044513e+07,0.700449,0.671836,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,0.783185,13.725374,18.116372,1014.300674,1352.042945,26.799279,19.803363,66.478178,2.248768,0.653326


In [11]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     404.787000
1     480.282000
2     579.482000
3     646.084000
4     590.339000
5     633.190143
6     592.014514
7     552.624343
8     473.789000
9     372.210000
10    409.903000
11    526.396000
12    511.494000
13    425.370000
14           NaN
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [12]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
0,-1.853967,0.985736,1.564403,-0.328372,-1.484238,-0.055169,1.204752,-0.990234,0.166091,-1.487840,...,-1.473167,-1.379428,-2.332251,-1.601508,2.921375,-0.953904,2.093748,1.992324,1.179694,-1.544673
1,-1.741077,1.046118,1.056829,-1.318724,-1.122666,0.411794,0.820657,-0.546688,0.428911,-1.378556,...,-1.014740,-1.366996,-1.506429,-1.744422,1.053586,0.562151,1.697171,1.627416,1.216235,-1.238766
2,-1.066520,1.247888,1.201979,0.704334,-0.826555,0.595931,0.275392,0.158672,0.847995,-1.195437,...,-0.672413,-1.381760,-0.525185,-1.295478,0.275310,-0.611236,0.967987,0.925970,1.059099,-0.954737
3,-0.511040,1.077778,1.033617,1.389206,-0.594315,0.501385,-0.132596,0.824346,1.230291,-0.920269,...,-0.443833,-1.386920,-0.109458,-0.492880,-0.122527,-1.536117,0.095293,0.256622,0.823156,-0.692862
4,-0.027298,0.830558,0.666485,1.712873,-0.440345,0.015898,-0.104860,1.148829,1.382743,-0.534838,...,-0.334747,0.118665,0.784642,-0.084132,-0.303621,-1.133132,-0.660911,-0.192930,0.663783,-0.453547
5,0.337205,0.334142,0.041545,1.232130,-0.340955,-0.476154,0.657046,0.830615,1.112297,-0.020933,...,-0.340807,0.402667,1.416245,0.430234,-0.429984,-0.839667,-0.599635,-0.090457,0.478422,-0.263819
6,0.607332,-0.138902,-0.476774,0.441307,-0.218082,-1.382832,1.615453,-0.215960,0.282052,0.569384,...,-0.419492,0.634325,0.894361,0.977646,-0.480884,0.064000,-0.529926,-0.323382,0.046150,0.006394
7,0.777643,-0.500010,-0.959834,-0.479181,-0.282585,-2.551876,0.558230,-0.940033,-0.632425,0.943972,...,-0.594885,0.840142,0.433894,1.095443,-0.460052,0.820856,-0.472756,-0.577001,-1.167704,0.105104
8,0.884898,-0.867272,-1.208523,-1.003804,1.315140,1.262203,-0.835563,-1.089341,-1.093812,1.140087,...,1.685884,0.971965,0.046269,0.978606,-0.460585,0.029259,-0.412203,-0.584859,-1.735304,1.060937
9,0.947050,-1.243880,-1.084715,-0.761339,1.367575,0.566995,-1.495939,-1.153056,-1.335633,1.102599,...,1.508921,0.888880,0.059226,0.852881,-0.586441,0.362748,-0.332495,-0.580810,-1.100296,1.264488


In [13]:
reshaped_train_input = dfToInputRNN(train_input)

In [14]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.74246201, -1.26271433, -0.80292463, -0.65577227,
          1.13768378,  0.45858597, -1.0453709 ,  1.21859064,
         -0.99976018,  0.79428371,  1.15575749, -0.52559858,
          1.30260559, -0.55253046, -0.25130359,  1.38787176,
          1.47154451, -1.80649616,  1.09296291,  0.94281138,
          0.52836084,  0.61258103,  0.87656257,  1.47801536,
          1.71982086,  1.28221181,  1.39086472,  1.58209844,
          1.08872091,  0.55212962, -1.81398118,  1.44113674,
          1.11191094,  1.30470835,  0.86390516, -1.67365968,
         -1.70738939, -1.55813196, -0.48498301, -0.75714639,
          0.40770893, -0.27394734,  2.02323219,  1.2908951 ,
          1.70465513,  1.81148084,  1.39968653, -1.36191442,
          1.28024593,  1.11054795,  0.12662799,  1.40414112,
          1.40468999,  1.45643959,  1.58509336,  1.32283157,
          1.30222047,  1.55138602,  1.91801602, -0.77489784,
          2.32098884, -1.1942541 , -1.17081042,  0.86269482,
          0.75907411,  0

In [15]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     404.787000
1     480.282000
2     579.482000
3     646.084000
4     590.339000
5     633.190143
6     592.014514
7     552.624343
8     473.789000
9     372.210000
10    409.903000
11    526.396000
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [16]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    526.396
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [17]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
12,0.398395,-0.999064,-0.787441,-1.136938,1.050925,-0.271725,1.108405,2.635823,-0.189959,0.726405,...,0.357602,0.721754,0.435649,1.373376,-0.585913,1.326673,-1.855589,-1.722974,-0.675291,1.2951


In [18]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.39839493, -0.99906413, -0.78744087, -1.13693763,
          1.05092544, -0.27172527,  1.10840491,  2.63582309,
         -0.18995911,  0.72640489,  2.66833308, -0.032632  ,
          0.98802435,  0.24980494, -0.08955808,  1.85434394,
          2.02706568, -2.21150703,  1.42938591,  0.85132668,
          0.14092386,  0.30902324,  1.00267904,  2.27970137,
          1.11272593,  0.61007902,  0.89064538,  1.42231438,
          1.02650562,  1.47127593, -1.53201473,  1.73503858,
          1.66610613,  1.0456894 ,  0.55289259, -1.81216593,
         -2.10342296, -2.55377478,  0.41940628, -1.75756445,
          0.241023  , -0.98659345,  1.52184916,  1.01398607,
          1.49290967,  1.40764092,  1.87930411, -2.51892072,
          2.83356214,  1.58620924, -0.02524128,  1.54231542,
          1.5176119 ,  1.7355269 ,  1.87941982,  1.54198129,
          1.51276712,  1.83795594,  2.62210693,  1.04993174,
          0.96476493, -1.48277861, -2.26044738,  0.35760229,
          0.72175409,  0

In [19]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    511.494
Name: Sergipe - Consumo de Cimento (t), dtype: float64

In [20]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [21]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [23]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[101315987, 108803925, 2375316620, 2552455697, 648253836, 2591512127, 2813930065, 220582004, 3733159360, 3012380942, 2770562723, 4150025588, 293366911, 1815195170, 1808878491, 3587853484, 2535534908, 2099652583, 1659478385, 991139197, 3680967248, 2189244850, 2176938241, 4070911534, 1800655404, 4106451328, 3777291386, 3318631305, 694368246, 3918703987, 2922602815, 1074781830, 808803521, 2372204831, 1213533241, 2125019132, 485360105, 2782293537, 2339145956, 2703727907, 2674362633, 3656307287, 315822324, 718505721, 3010348872, 3178233402, 1304201244, 3407288200, 1621281044, 3885554963, 1871572157, 4115695982, 2244196621, 704968169, 710523618, 3141259471, 254202212, 2456411412, 4220098714, 2339230504, 4127489212, 1898972968, 1499897114, 393235378, 629119127, 4119871508, 2203107368, 1990845534, 2773957774, 3661672032, 3405819480, 1764080375, 4037584086, 1613439265, 1270377994, 2469208414, 209660034, 1746124721, 2787221097, 3936171290, 4043782780, 2693833786, 3424183997, 1174714134, 17887416

loss: 76.6299057006836


Step: 88 ___________________________________________
loss: 60.68385314941406


Step: 89 ___________________________________________
loss: 79.30025482177734


Step: 90 ___________________________________________
loss: 43.34220886230469


Step: 91 ___________________________________________
loss: 60.96501159667969


Step: 92 ___________________________________________
loss: 63.35316467285156


Step: 93 ___________________________________________
loss: 70.6656723022461


Step: 94 ___________________________________________
loss: 43.81242752075195


Step: 95 ___________________________________________
loss: 61.414554595947266


Step: 96 ___________________________________________
loss: 52.365535736083984


Step: 97 ___________________________________________
loss: 60.29911422729492


Step: 98 ___________________________________________
loss: 72.35965728759766


Step: 99 ___________________________________________
loss: 64.36235809326172


final_seed: 209660034


In [24]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 280100.6562
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 273910.5000
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 263031.1562
Epoch 4/10000
1/1 [==============================] - 0s 8ms/step - loss: 207160.7656
Epoch 5/10000
1/1 [==============================] - 0s 7ms/step - loss: 22381.3184
Epoch 6/10000
1/1 [==============================] - 0s 8ms/step - loss: 11899.0244
Epoch 7/10000
1/1 [==============================] - 0s 7ms/step - loss: 8112.5879
Epoch 8/10000
1/1 [==============================] - 0s 5ms/step - loss: 9117.3477
Epoch 9/10000
1/1 [==============================] - 0s 8ms/step - loss: 7824.8022
Epoch 10/10000
1/1 [==============================] - 0s 8ms/step - loss: 5220.3335
Epoch 11/10000
1/1 [==============================] - 0s 5ms/step - loss: 5974.8594
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - loss: 52

1/1 [==============================] - 0s 448ms/step - loss: 1037.3514
Epoch 99/10000
1/1 [==============================] - 0s 11ms/step - loss: 1132.0673
Epoch 100/10000
1/1 [==============================] - 0s 7ms/step - loss: 1353.2552
Epoch 101/10000
1/1 [==============================] - 0s 22ms/step - loss: 468.0272
Epoch 102/10000
1/1 [==============================] - 0s 5ms/step - loss: 812.8188
Epoch 103/10000
1/1 [==============================] - 0s 5ms/step - loss: 759.2618
Epoch 104/10000
1/1 [==============================] - 0s 5ms/step - loss: 711.6016
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 581.2283
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 1172.3119
Epoch 107/10000
1/1 [==============================] - 0s 6ms/step - loss: 1115.3208
Epoch 108/10000
1/1 [==============================] - 0s 5ms/step - loss: 1011.5312
Epoch 109/10000
1/1 [==============================] - 0s 8ms/step - loss: 165.8401

1/1 [==============================] - 0s 4ms/step - loss: 969.7617
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 576.3492
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 795.2877
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 428.4252
Epoch 199/10000
1/1 [==============================] - 0s 5ms/step - loss: 656.7866
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 502.5022
Epoch 201/10000
1/1 [==============================] - 0s 5ms/step - loss: 565.9532
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - loss: 444.0879
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - loss: 821.0175
Epoch 204/10000
1/1 [==============================] - 0s 5ms/step - loss: 727.8769
Epoch 205/10000
1/1 [==============================] - 0s 5ms/step - loss: 327.8153
Epoch 206/10000
1/1 [==============================] - 0s 4ms/step - loss: 930.6351
Epoch 20

1/1 [==============================] - 0s 4ms/step - loss: 434.7555
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 836.5357
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 349.2607
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 563.5472
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 419.6767
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 338.8459
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 606.9069
Epoch 299/10000
1/1 [==============================] - 0s 4ms/step - loss: 363.9276
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 455.3881
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 794.6136
Epoch 302/10000
1/1 [==============================] - 0s 4ms/step - loss: 537.8129
Epoch 303/10000
1/1 [==============================] - 0s 4ms/step - loss: 244.8618
Epoch 30

1/1 [==============================] - 0s 5ms/step - loss: 212.7563
Epoch 390/10000
1/1 [==============================] - 0s 5ms/step - loss: 404.3968
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 704.3228
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 410.9081
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - loss: 330.0049
Epoch 394/10000
1/1 [==============================] - 0s 5ms/step - loss: 545.5388
Epoch 395/10000
1/1 [==============================] - 0s 5ms/step - loss: 801.5231
Epoch 396/10000
1/1 [==============================] - 0s 4ms/step - loss: 452.9044
Epoch 397/10000
1/1 [==============================] - 0s 5ms/step - loss: 300.8437
Epoch 398/10000
1/1 [==============================] - 0s 6ms/step - loss: 730.1865
Epoch 399/10000
1/1 [==============================] - 0s 4ms/step - loss: 692.2165
Epoch 400/10000
1/1 [==============================] - 0s 5ms/step - loss: 557.3698
Epoch 40

...

1/1 [==============================] - 0s 4ms/step - loss: 317.3492
Epoch 1836/10000
1/1 [==============================] - 0s 4ms/step - loss: 754.7605
Epoch 1837/10000
1/1 [==============================] - 0s 4ms/step - loss: 369.1287
Epoch 1838/10000
1/1 [==============================] - 0s 4ms/step - loss: 370.6123
Epoch 1839/10000
1/1 [==============================] - 0s 4ms/step - loss: 776.9738
Epoch 1840/10000
1/1 [==============================] - 0s 4ms/step - loss: 572.5725
Epoch 1841/10000
1/1 [==============================] - 0s 4ms/step - loss: 317.4583
Epoch 1842/10000
1/1 [==============================] - 0s 4ms/step - loss: 489.4141
Epoch 1843/10000
1/1 [==============================] - 0s 4ms/step - loss: 329.2553
Epoch 1844/10000
1/1 [==============================] - 0s 4ms/step - loss: 399.8770
Epoch 1845/10000
1/1 [==============================] - 0s 4ms/step - loss: 364.7576
Epoch 1846/10000
1/1 [==============================] - 0s 4ms/step - loss: 211.31

1/1 [==============================] - 0s 5ms/step - loss: 223.5685
Epoch 1932/10000
1/1 [==============================] - 0s 5ms/step - loss: 341.9761
Epoch 1933/10000
1/1 [==============================] - 0s 5ms/step - loss: 117.3406
Epoch 1934/10000
1/1 [==============================] - 0s 5ms/step - loss: 243.6572
Epoch 1935/10000
1/1 [==============================] - 0s 5ms/step - loss: 212.3598
Epoch 1936/10000
1/1 [==============================] - 0s 5ms/step - loss: 343.0619
Epoch 1937/10000
1/1 [==============================] - 0s 5ms/step - loss: 420.5639
Epoch 1938/10000
1/1 [==============================] - 0s 5ms/step - loss: 93.0619
Epoch 1939/10000
1/1 [==============================] - 0s 5ms/step - loss: 191.1610
Epoch 1940/10000
1/1 [==============================] - 0s 4ms/step - loss: 232.8021
Epoch 1941/10000
1/1 [==============================] - 0s 4ms/step - loss: 110.9758
Epoch 1942/10000
1/1 [==============================] - 0s 5ms/step - loss: 193.032

1/1 [==============================] - 0s 4ms/step - loss: 513.2896
Epoch 2028/10000
1/1 [==============================] - 0s 5ms/step - loss: 327.9290
Epoch 2029/10000
1/1 [==============================] - 0s 5ms/step - loss: 383.1446
Epoch 2030/10000
1/1 [==============================] - 0s 5ms/step - loss: 160.6557
Epoch 2031/10000
1/1 [==============================] - 0s 5ms/step - loss: 304.0665
Epoch 2032/10000
1/1 [==============================] - 0s 5ms/step - loss: 868.7036
Epoch 2033/10000
1/1 [==============================] - 0s 5ms/step - loss: 297.1118
Epoch 2034/10000
1/1 [==============================] - 0s 5ms/step - loss: 315.2516
Epoch 2035/10000
1/1 [==============================] - 0s 4ms/step - loss: 365.4374
Epoch 2036/10000
1/1 [==============================] - 0s 4ms/step - loss: 163.7882
Epoch 2037/10000
1/1 [==============================] - 0s 4ms/step - loss: 311.7921
Epoch 2038/10000
1/1 [==============================] - 0s 4ms/step - loss: 353.10

1/1 [==============================] - 0s 4ms/step - loss: 660.9854
Epoch 2124/10000
1/1 [==============================] - 0s 5ms/step - loss: 686.2148
Epoch 2125/10000
1/1 [==============================] - 0s 5ms/step - loss: 144.8466
Epoch 2126/10000
1/1 [==============================] - 0s 5ms/step - loss: 162.5126
Epoch 2127/10000
1/1 [==============================] - 0s 5ms/step - loss: 286.1488
Epoch 2128/10000
1/1 [==============================] - 0s 4ms/step - loss: 414.7170
Epoch 2129/10000
1/1 [==============================] - 0s 4ms/step - loss: 392.0049
Epoch 2130/10000
1/1 [==============================] - 0s 5ms/step - loss: 526.5547
Epoch 2131/10000
1/1 [==============================] - 0s 4ms/step - loss: 445.2695
Epoch 2132/10000
1/1 [==============================] - 0s 4ms/step - loss: 199.8840
Epoch 2133/10000
1/1 [==============================] - 0s 5ms/step - loss: 236.5850
Epoch 2134/10000
1/1 [==============================] - 0s 4ms/step - loss: 321.66

1/1 [==============================] - 0s 5ms/step - loss: 409.2476
Epoch 2220/10000
1/1 [==============================] - 0s 4ms/step - loss: 408.0196
Epoch 2221/10000
1/1 [==============================] - 0s 4ms/step - loss: 378.7070
Epoch 2222/10000
1/1 [==============================] - 0s 4ms/step - loss: 822.2195
Epoch 2223/10000
1/1 [==============================] - 0s 4ms/step - loss: 536.2259
Epoch 2224/10000
1/1 [==============================] - 0s 5ms/step - loss: 241.7826
Epoch 2225/10000
1/1 [==============================] - 0s 4ms/step - loss: 390.8966
Epoch 2226/10000
1/1 [==============================] - 0s 5ms/step - loss: 433.9865
Epoch 2227/10000
1/1 [==============================] - 0s 5ms/step - loss: 376.4851
Epoch 2228/10000
1/1 [==============================] - 0s 4ms/step - loss: 291.3253
Epoch 2229/10000
1/1 [==============================] - 0s 4ms/step - loss: 424.8748
Epoch 2230/10000
1/1 [==============================] - 0s 4ms/step - loss: 763.48

In [25]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 679ms/step
[511.494](test_target) - [[519.07294]](prediction) = 7.578937011718722


In [26]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.014817254966272765

In [27]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [28]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
0,-0.863219,-0.959139,1.358624,-0.017092,-1.263295,-1.359965,1.148205,-1.122043,-1.121418,-1.109924,...,-1.279461,-0.519495,-1.188168,-0.290340,1.355077,-0.954425,1.088406,1.078152,0.417308,-1.239600
1,-0.538514,-0.420452,-1.019320,-1.216109,0.081135,0.344018,0.140893,-0.184474,-0.185492,-0.204018,...,0.117968,1.398867,-0.070139,-1.053486,-0.327076,1.380985,0.237800,0.253510,0.961556,0.030272
2,1.401733,1.379591,-0.339304,1.233202,1.182160,1.015947,-1.289098,1.306518,1.306910,1.313942,...,1.161492,-0.879372,1.258307,1.343826,-1.028001,-0.426561,-1.326206,-1.331661,-1.378864,1.209328


train_target:


,Sergipe - Consumo de Cimento (t)
0,404.787
1,480.282
2,579.482


test_input:


...

,Sergipe - PIB - Estadual,Sergipe - PIB - Construção Civil,Sergipe - PIB - Per Capita,Sergipe - PIB - Preços de Mercado,Sergipe - IDH,Sergipe - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Sergipe - IDH Longevidade,Sergipe - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Sergipe - IDH Educacao
12,0.398395,-0.999064,-0.787441,-1.136938,1.050925,-0.271725,1.108405,2.635823,-0.189959,0.726405,...,0.357602,0.721754,0.435649,1.373376,-0.585913,1.326673,-1.855589,-1.722974,-0.675291,1.2951


test_target:


,Sergipe - Consumo de Cimento (t)
12,511.494


1/1 [==============================] - 1s 573ms/step
Error: 7.578937011718722




[584.9185180664062,
 658.4121704101562,
 542.6326904296875,
 607.9905395507812,
 586.2551879882812,
 545.0480346679688,
 477.4184265136719,
 368.1544189453125,
 407.12939453125,
 519.0729370117188]

In [29]:
display(targets)
display(predictions)

[646.0840000000001,
 590.339,
 633.1901428571429,
 592.0145142857143,
 552.6243428571429,
 473.789,
 372.21,
 409.903,
 526.396,
 511.494]

[584.9185180664062,
 658.4121704101562,
 542.6326904296875,
 607.9905395507812,
 586.2551879882812,
 545.0480346679688,
 477.4184265136719,
 368.1544189453125,
 407.12939453125,
 519.0729370117188]

In [30]:
mae = mean_absolute_error(predictions, targets)
mae

61.446455988420745

In [31]:
porcentage = mae/np.mean(targets)
porcentage

0.1157610147700749